## Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados `AVI SSVEP Single Target`. 

### Passos para a realização da classificação:

1. **Carrega** o arquivo `fif` (`mne.EpochsArray`) dos dados **filtrados**;
2. **Determine o limiar** para isolar cada uma das frequências estimuladas. Por exemplo, a faixa de frequência para o estímulo de 6.5 Hz irá resultar em pontos (`PSD`) que irão variar de 6.3 à 6.7 Hz, caso o limiar seja de 0.2 Hz;
3. **Obter a "energia"** do sinal por meio do cálculo `compute_psd` para cada uma das faixas de frequência que podem ser estimuladas. Por exemplo:
    - Obtenha todas as frequências estimuladas. Ex: 6, 6.5, 7, 7.5, 8.2 e 9.3;
    - Obtenha o valor mínimo e o máximo para cada frequência utilizando limiar. Ex: (5.8, 6.2), (6.3, 6.7), ...
    - Aplique o `compute_psd` para cada tupla (min, max), por meio dos parâmetros `fmin` e `fmax` do mesmo método.
4. Com as listas de pontos isoladas e computadas (`PSD`) para cada amostra, aplique um cálculo de característica adequada. Características manuais interessantes para este exemplo podem ser `max_value`, `average` ou `median`. No fim deste passo iremos obter um **vetor de características**;
5. Por fim, realize a **classificação**, que será um **cálculo de voto** simples (maior valor é provavelmente o a frequência evocada).

A seguir, um exemplo desta classificação com os dados `single target` de `AVI dataset`:

In [146]:
import mne
import numpy as np
import warnings
warnings.filterwarnings('ignore')

data = mne.read_epochs("./mne_data.fif")
data

Reading /home/vinicius/Documents/rp/RP_SSVEP/ssvep/src/mne_data.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   29998.05 ms
        0 CTF compensation matrices available
Not setting metadata
84 matching events found
No baseline correction applied
0 projection items activated


Number of events,84
Events,10: 126: 126.5: 127: 127.5: 128.2: 129.3: 12
Time range,0.000 – 29.998 s
Baseline,off


In [147]:
y = np.load("../datasets/avi/single/labels_full.npy")
lista_y = []

for i in y:
    for j in i:
        lista_y.append(j[0])

y_set = sorted(set(lista_y))
aux = y_set.pop()
y_set.insert(0, aux)
print(y_set)

[10.0, 6.0, 6.5, 7.0, 7.5, 8.2, 9.3]


In [148]:
limiar = 0.2
acertos = 0

for i in range(len(data)):

    dicionario_y = {str(value): index for index, value in enumerate(y_set)}

    for frequencia in y_set:
        freq_max = frequencia + limiar
        freq_min = frequencia - limiar

        view = data[i].compute_psd(method='multitaper', fmin=freq_min, fmax=freq_max, verbose=False)

        lista_valores = np.array((view)[0][0])

        valores_max = np.max(lista_valores)

        dicionario_y[str(frequencia)] = valores_max

    target_freq = max(dicionario_y, key=dicionario_y.get)
    
    if float(target_freq) == float(list(data[i].event_id)[0]):
        acertos += 1

In [149]:
# y = np.load("../datasets/avi/single/labels_full.npy")
precisao = (acertos * 100) / len(lista_y)
print(f"Precisao: {precisao:.2f}")
print(f"Acertos: {acertos}")

Precisao: 86.90
Acertos: 73
